In [ ]:
# original lane mask prediction
# try weighted model
# Lane marking only
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"   # trying disabling GPU

import tensorflow as tf
import segmentation_models as sm
import time
import cv2
import random

import numpy as np
from matplotlib import pyplot as plt 
from keras import backend as K

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

SIZE_X = 640
SIZE_Y = 480

validation_portion = 0.2
sample_size_limit = 10_000

sem_path = 'c:/SelfDrive/Semantic Segmentation/out_sem/rgb'
map_path = 'c:/SelfDrive/Semantic Segmentation/out_sem/map_ln'

'''
def class_weighted_cross_entropy_loss(y_true,y_pred):
    class_weights = np.array([0.98,0.02])
    cross_entropy_loss = tf.keras.losses.binary_crossentropy(y_true,y_pred,from_logits=False)
    class_weights = tf.cast(tf.reshape(class_weights,[1,-1]), tf.float32)
    tf.cast(cross_entropy_loss, tf.float32)
    weighterd_cross_entropy_loss = tf.reduce_mean(cross_entropy_loss * class_weights)
    return weighterd_cross_entropy_loss
'''

def weighted_binary_crossentropy( y_true, y_pred, weight=1. ) :
    y_true = K.clip(y_true, K.epsilon(), 1-K.epsilon())
    y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())
    logloss = -(y_true * K.log(y_pred) * weight + (1 - y_true) * K.log(1 - y_pred))
    return K.mean( logloss, axis=-1)

sem_images = os.listdir(sem_path)
random.shuffle(sem_images)

#reduce number of images here
sem_images = sem_images[0:sample_size_limit]

img_count = len(sem_images)

train_len = int(img_count * (1-validation_portion))
val_len = img_count - train_len

x_train = np.zeros(shape=(train_len,SIZE_Y,SIZE_X,3),dtype=np.float16)
x_val = np.zeros(shape=(val_len,SIZE_Y,SIZE_X,3),dtype=np.float16)

y_train =  np.zeros(shape=(train_len,SIZE_Y,SIZE_X),dtype=np.float16)
y_val =  np.zeros(shape=(val_len,SIZE_Y,SIZE_X),dtype=np.float16)

i = 0
i_val = 0
for file in sem_images:
        pth = os.path.join(sem_path,file) 
        img = cv2.imread(pth,cv2.IMREAD_COLOR)
        pth = os.path.join(map_path,file) 
        map = cv2.imread(pth,0)
        
        if i < train_len:
            x_train[i] = img/255
            y_train[i] = map/255
        else:
            x_val[i_val] = img/255
            y_val[i_val] = map/255
            i_val += 1
        
        i += 1
        
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

# run without GPU
NAME = "UNET_LANES_WCE_LOSS-{}".format(int(time.time()))
#tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
model.compile(optimizer='adam',loss=weighted_binary_crossentropy,metrics=['mse'])
#model.summary()


history = model.fit(x_train,y_train,batch_size=16,epochs=20,verbose=1,validation_data=(x_val,y_val))
model.save('model/{}'.format(NAME),save_format='tf')

In [1]:
# testing cropping 
import os
import cv2

BOTTOM_CROP = 0.55 #take bottom share oh sem image
sem_path = 'c:/SelfDrive/Map Projection/img'
sem_images = os.listdir(sem_path)

# get sizing
test_file = sem_images[0]
test_img = cv2.imread(os.path.join(sem_path,test_file),cv2.IMREAD_COLOR)
SIZE_Y = test_img.shape[0]
SIZE_X = test_img.shape[1]

cropped_img = test_img[int(SIZE_Y*(1-BOTTOM_CROP)):,:,:]
#re-define height after cropping
SIZE_Y = cropped_img.shape[0]
cv2.imshow("test", cropped_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# changing original model
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"   # trying disabling GPU

import tensorflow as tf
import time
import cv2
import random
from tensorflow import keras

import numpy as np
from matplotlib import pyplot as plt 
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Resizing, UpSampling2D, Dropout, MaxPooling2D, Conv2D, Conv2DTranspose, Concatenate, Embedding, Reshape, Flatten, Activation, BatchNormalization
from keras.optimizers import SGD
from keras import regularizers


BATCH_SIZE = 4

#SIZE_X = 640
#SIZE_Y = 480
BOTTOM_CROP = 0.55 #take bottom share oh sem image

validation_portion = 0.2
sample_size_limit = 10_000

sem_path = 'c:/SelfDrive/Map Projection/img'
map_path = 'c:/SelfDrive/Map Projection/map_clean'

def weighted_binary_crossentropy( y_true, y_pred, weight=1. ) :
    y_true = K.clip(y_true, K.epsilon(), 1-K.epsilon())
    y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())
    logloss = -(y_true * K.log(y_pred) * weight + (1 - y_true) * K.log(1 - y_pred))
    return K.mean( logloss, axis=-1)

# old sem_images = os.listdir(sem_path)
sem_images = [os.path.join(sem_path, file) for file in os.listdir(sem_path) if file.endswith('.png')]
map_images = [os.path.join(map_path, file) for file in os.listdir(map_path) if file.endswith('.png')]
random.shuffle(sem_images)

# get sizing
test_file = sem_images[0]
test_img = cv2.imread(test_file,cv2.IMREAD_COLOR)
size_y = test_img.shape[0]
size_x = test_img.shape[1]

cropped_img = test_img[int(size_y*(1-BOTTOM_CROP)):,:,:]
#re-define height after cropping
size_y = cropped_img.shape[0]

test_file = map_images[0]
test_map = cv2.imread(test_file,cv2.IMREAD_COLOR)
size_y_map = test_map.shape[0]
size_x_map = test_map.shape[1]


# Create a custom data generator
def custom_data_generator(image_files, batch_size):
    num_samples = len(image_files)
    while True:
        indices = np.random.randint(0, num_samples, batch_size)
        batch_images = []
        batch_maps = []
        for idx in indices:
            image_path = image_files[idx]
            map_p = os.path.join(map_path,os.path.basename(image_path))
            image = preprocess_image(image_path)
            map_img = preprocess_map(map_p)
            batch_images.append(image)
            batch_maps.append(map_img)
        yield [np.array(batch_images),batch_maps]

def preprocess_image(image_path):
    image = keras.preprocessing.image.load_img(image_path, target_size=(size_x,size_y)) #where image_size = (WIDTH,HEIGHT)
    image = keras.preprocessing.image.img_to_array(image)
    #bottom only part of image if it is a semantic or RGB image
    image = image[int(image.shape[0]*(1-BOTTOM_CROP)):,:,:]
    image = image / 255.0  # Normalize pixel values between 0 and 1
    return image

def preprocess_map(map_pth):
    map_im = keras.preprocessing.image.load_img(map_pth, target_size=(size_x_map,size_y_map)) #where image_size = (WIDTH,HEIGHT)
    map_im = keras.preprocessing.image.img_to_array(map_im)
    map_im = map_im / 255.0  # Normalize pixel values between 0 and 1
    return map_im


def create_model():
    # Image input
    output_shape = (size_y_map,size_x_map,3)
    image_input = Input(shape=(size_y, size_x, 3))
    # Preprocess the image input
    x = Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same')(image_input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu',padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(256, kernel_size=(3, 3), activation='relu',padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    #x = Dense(256, activation='relu',activity_regularizer=regularizers.L2(1e-5))(x)
    x = Dropout(0.2)(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same') (x)
    x = Resizing(264,240) (x)
    output = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    #x = Dense(128, activation='relu',activity_regularizer=regularizers.L2(1e-5))(x)
        # Dense layers for prediction
    # Create the model
    #x = Conv2DTranspose(3, (3, 3), strides=(8, 3), padding='same', activation='sigmoid')(x)
    #output = Reshape(target_shape=(size_y_map, size_x_map, 3)) (x)
    #model = Model(inputs=image_input, outputs=output)
    
    
    model = Model(image_input, output)
    model.build((None,) + output_shape)
    return model


# Split the data into training and validation sets
split_index = int(len(sem_images) * 0.8)  # 80% for training, 20% for validation
train_files, val_files = sem_images[:split_index], sem_images[split_index:]

# Create data generators for training and validation
train_generator = custom_data_generator(train_files, BATCH_SIZE)
val_generator = custom_data_generator(val_files, BATCH_SIZE)


# run 
NAME = "UNET_MAP_WCE_LOSS-{}".format(int(time.time()))
#tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
# need to build a new model

model = create_model()
model.compile(optimizer='adam',loss=weighted_binary_crossentropy,metrics=['mse'])

model.summary()

for layer in model.layers:
    print(layer.output_shape)
# Train the model
history = model.fit(train_generator, steps_per_epoch=len(train_files) // BATCH_SIZE, epochs=15,
          validation_data=val_generator, validation_steps=len(val_files) // BATCH_SIZE)


model.save('model/{}'.format(NAME),save_format='tf')

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 265, 640, 3)]     0         
                                                                 
 conv2d_21 (Conv2D)          (None, 265, 640, 64)      1792      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 132, 320, 64)     0         
 2D)                                                             
                                                                 
 conv2d_22 (Conv2D)          (None, 132, 320, 128)     73856     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 66, 160, 128)     0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 66, 160, 256)      2951

InvalidArgumentError: Graph execution error:

Detected at node 'mean_squared_error/SquaredDifference' defined at (most recent call last):
    File "c:\Users\Vadim\.conda\envs\tf2\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "c:\Users\Vadim\.conda\envs\tf2\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "c:\Users\Vadim\.conda\envs\tf2\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Vadim\.conda\envs\tf2\lib\site-packages\traitlets\config\application.py", line 1041, in launch_instance
      app.start()
    File "c:\Users\Vadim\.conda\envs\tf2\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\Vadim\.conda\envs\tf2\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Vadim\.conda\envs\tf2\lib\asyncio\base_events.py", line 541, in r

In [4]:
test_map.shape

(264, 240, 3)